In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Venks\\Desktop\\Project\\MLOPS_Bootcamp\\Loan_Risk_Assessment_Platform\\research'

In [5]:
os.chdir("../")

In [3]:
import pandas as pd

In [7]:
data = pd.read_csv("artifacts/data_ingestion/emi_prediction_dataset.csv")
data.head()

C:\Users\Venks\AppData\Local\Temp\ipykernel_30000\2866169937.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("artifacts/data_ingestion/emi_prediction_dataset.csv")


,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38.0,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38.0,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38.0,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58.0,Female,Married,High School,66800.0,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
4,48.0,Female,Married,Professional,57300.0,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  object 
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               402396 non-null  object 
 4   monthly_salary          404800 non-null  object 
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            402374 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     

Entity- Is something we have to change as per our problem statement. Status File True/False checks if the schema(features) matches with the downloaded dataset schema to proceed with training

In [10]:
data.columns

Index(['age', 'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'emi_eligibility',
       'max_monthly_emi'],
      dtype='object')

In [11]:
data.dtypes

age                        object
gender                     object
marital_status             object
education                  object
monthly_salary             object
employment_type            object
years_of_employment       float64
company_type               object
house_type                 object
monthly_rent              float64
family_size                 int64
dependents                  int64
school_fees               float64
college_fees              float64
travel_expenses           float64
groceries_utilities       float64
other_monthly_expenses    float64
existing_loans             object
current_emi_amount        float64
credit_score              float64
bank_balance               object
emergency_fund            float64
emi_scenario               object
requested_amount          float64
requested_tenure            int64
emi_eligibility            object
max_monthly_emi           float64
dtype: object

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    cleaned_file_path: str


Config Manager: Constructor/ Magic method takes in self, filepaths of config, params and schema. It has one more method get_data_validation_config which returns the entity defined earlier

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            cleaned_file_path=config.cleaned_file_path
        )

        return data_validation_config

Data Validation component: 

In [21]:
import os
from mlProject import logger

In [23]:
import pandas as pd
import numpy as np
from mlProject import logger

class DataValidation:
    def __init__(self, config):
        self.config = config

    def validate_column_types(self, data):
        schema_types = self.config.all_schema
        mismatches = []
        for col, dtype in schema_types.items():
            if col not in data.columns:
                mismatches.append((col, "missing", dtype))
            elif str(data[col].dtype) != dtype:
                mismatches.append((col, str(data[col].dtype), dtype))
        return mismatches

    def summarize_uniques(self, data):
        summary = {}
        for col in data.columns:
            vals = np.unique(data[col].astype(str))
            nr_values = len(vals)
            if nr_values <= 45:
                summary[col] = f"{nr_values}: {vals.tolist()}"
            else:
                summary[col] = f"{nr_values} unique"
        return summary

    def convert_numerics(self, data):
        for col in ['age', 'monthly_salary', 'bank_balance']:
            if col in data.columns:
                data[col] = pd.to_numeric(data[col], errors='coerce')
        logger.info("✓ Data type conversion completed")
        return data

    def check_missing(self, data):
        missing_data = data.isnull().sum()
        missing_percent = (missing_data / len(data)) * 100
        missing_df = pd.DataFrame({'Missing_Count': missing_data, 'Percentage': missing_percent})
        logger.info(f"Missing data summary:\n{missing_df[missing_df['Missing_Count'] > 0]}")
        return missing_df

    def standardize_categories(self, data):
        gender_map = {'F': 'Female', 'FEMALE': 'Female', 'Female': 'Female', 
                      'M': 'Male', 'MALE': 'Male', 'Male': 'Male',
                      'female': 'Female', 'male': 'Male'}
        if 'gender' in data.columns:
            data['gender'] = data['gender'].map(gender_map)
        edu_map = {'Professional': 'Professional', 'Post Graduate': 'Post Graduate',
                   'Graduate': 'Graduate', 'High School': 'High School'}
        if 'education' in data.columns:
            data['education'] = data['education'].map(edu_map).fillna(data['education'].mode()[0])
        for col in ['employment_type', 'company_type', 'house_type', 'marital_status', 'emi_scenario']:
            if col in data.columns:
                data[col] = data[col].astype(str).str.title()
        logger.info("✓ Category standardization completed")
        return data

    def impute_missing(self, data):
        num_cols_with_na = ['age','monthly_salary','monthly_rent', 'credit_score', 'bank_balance', 'emergency_fund']
        cat_cols_with_na = ['education']
        for col in num_cols_with_na:
            if col in data.columns:
                median_val = data[col].median()
                data[col].fillna(median_val, inplace=True)
        for col in cat_cols_with_na:
            if col in data.columns:
                mode_val = data[col].mode()[0]
                data[col].fillna(mode_val, inplace=True)
        logger.info("✓ Missing value imputation completed")
        return data

    def run_full_validation(self):
        status_report = {}
        data = pd.read_csv(self.config.unzip_data_dir)

        # 1. Column type validation
        col_type_mismatches = self.validate_column_types(data)
        status_report['column_types'] = col_type_mismatches
        status_report['column_types_msg'] = (
            "Column validation passed" if not col_type_mismatches else f"Type mismatches: {col_type_mismatches}"
        )

        # 2. Check unique values for all features
        uniques = self.summarize_uniques(data)
        status_report['uniques'] = uniques

        # 3. Data type conversions
        data = self.convert_numerics(data)

        # 4. Missing summary before imputation
        missing_before = self.check_missing(data)
        status_report['missing_before'] = missing_before

        # 5. Categorical standardization
        data = self.standardize_categories(data)
        status_report['category_standardization_msg'] = "Category standardization complete"

        # 6. Imputation for missing values
        data = self.impute_missing(data)
        missing_after = self.check_missing(data)
        status_report['missing_after'] = missing_after

        status_report["missing_msg"] = (
            "All missing values handled" if missing_after['Missing_Count'].sum() == 0 else "Missing values remain"
        )

        # 7. Info and value check of main categoricals
        status_report['data_info'] = str(data.info())
        for col in data.select_dtypes(include='object').columns:
            status_report[col+'_final_values'] = data[col].unique().tolist()

        # 8. Save cleaned data to correct artifacts location
        data.to_csv(self.config.cleaned_file_path, index=False)
        logger.info("✓ Final cleaned data saved for EDA and pipeline progression")

        # 9. Final status log
        status_report['final_status'] = "SUCCESS" if (
            not col_type_mismatches and missing_after['Missing_Count'].sum() == 0
        ) else "FAIL"
        status_report['final_status_msg'] = (
            "Validation and preprocessing successful!"
            if status_report['final_status'] == "SUCCESS"
            else "Check type/missing value errors above."
        )

        # Save status to validation status file
        with open(self.config.STATUS_FILE, 'w') as f:
            for key, val in status_report.items():
                f.write(f"{key}: {val}\n")
        return status_report


Pipeline

In [24]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    validation_status = data_validation.run_full_validation()
    print("Validation report written. Status:", validation_status.get('final_status', 'UNKNOWN'))
except Exception as e:
    print("Error in data validation:", str(e))
    raise e


[2025-11-12 18:26:48,501: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-12 18:26:48,508: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-12 18:26:48,511: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-12 18:26:48,514: INFO: common: created directory at: artifacts]
[2025-11-12 18:26:48,514: INFO: common: created directory at: artifacts/data_validation]


C:\Users\Venks\AppData\Local\Temp\ipykernel_30000\741570855.py:76: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.config.unzip_data_dir)


[2025-11-12 18:26:56,910: INFO: 741570855: ✓ Data type conversion completed]
[2025-11-12 18:26:57,014: INFO: 741570855: Missing data summary:
                Missing_Count  Percentage
age                         3    0.000741
education                2404    0.593874
monthly_salary           1993    0.492342
monthly_rent             2426    0.599308
credit_score             2420    0.597826
bank_balance             4392    1.084980
emergency_fund           2351    0.580781]
[2025-11-12 18:26:57,315: INFO: 741570855: ✓ Category standardization completed]
[2025-11-12 18:26:57,384: INFO: 741570855: ✓ Missing value imputation completed]
[2025-11-12 18:26:57,470: INFO: 741570855: Missing data summary:
Empty DataFrame
Columns: [Missing_Count, Percentage]
Index: []]


C:\Users\Venks\AppData\Local\Temp\ipykernel_30000\741570855.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(median_val, inplace=True)
C:\Users\Venks\AppData\Local\Temp\ipykernel_30000\741570855.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  float64
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               404800 non-null  object 
 4   monthly_salary          404800 non-null  float64
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            404800 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     